In [127]:
import pandas as p

In [128]:
restaurantes = p.read_json("rest.txt")

In [129]:
restaurantes

,Nome,Cozinha,Endereco,Valor
0,El Carreiro,"Argentina, Parrilla","Alameda Dilson Jatahy Fonseca, 2150 Stella Mar...","Menu +Plus - Almoço: R$ 73,90 - Jantar: R$ 94,90"
1,La Lupa,"Italiana, Mediterrânea","Rua Direita do Santo Antônio, 491 Santo Antôni...","Menu +Plus - Almoço: R$ 73,90 - Jantar: R$ 94,90"
2,Cedro - Quality Hotel e Suítes São Salvador,Contemporânea,"Rua Jose Peroba, 244 Stiep - Salvador - BA","Menu +Plus - Almoço: R$ 73,90 - Jantar: R$ 94,90"
3,Canto Restaurante,Internacional,"Travessa Prudente de Moraes, 65 Rio Vermelho -...","Menu Premium - Almoço: R$ 95,00 - Jantar: R$ 1..."
4,Joca Mesa Bar,"Contemporânea, Regional","Rua Direita de Santo Antônio, 90 Santo Antônio...","Menu +Plus - Almoço: R$ 73,90 - Jantar: R$ 94,90"
...,...,...,...,...
458,Pereira Restaurante - Porto da Barra,Contemporânea,"Av. Sete de Setembro, 3959 Barra - Salvador - BA","Menu Premium - Almoço: R$ 95,00 - Jantar: R$ 1..."
459,Pasta em Casa - Alphaville,Italiana,"Avenida Alphaville, 52 Alphaville I - Salvador...","Menu Premium - Almoço: R$ 95,00 - Jantar: R$ 1..."
460,Cantina Volpi - Paralela,Italiana,"Avenida Alphaville, 7417 Alphaville I - Salvad...","Menu +Plus - Almoço: R$ 73,90"
461,Seven Wonders Café - Spot Barra,,"Avenida Oceânica, 859 Barra - Salvador - BA","Menu RW - Almoço: R$ 59,90"


In [130]:
restaurantes["Cozinha1"] = restaurantes["Cozinha"].fillna('').str.split(',', n=1).str[0]

In [131]:
restaurantes["Cozinha2"] = restaurantes["Cozinha"].fillna('').str.split(',', n=1).str[1]

In [132]:
restaurantes["Bairro"] = restaurantes["Endereco"].fillna('').str.split(', ', n=1).str[1]

In [133]:
restaurantes["Bairro"] = restaurantes["Bairro"].fillna('').str.split(' -', n=1).str[0]

In [134]:
restaurantes["Bairro"] = restaurantes["Bairro"].fillna('').str.split(' ', n=1).str[1]

In [135]:
# restaurantes["Bairro"] = restaurantes["Bairro"].fillna('').str.replace(r'[0-9]', '', regex=True)

In [136]:
restaurantes["Almoco"] = restaurantes["Valor"].fillna('').str.split('- Almoço: ',n=1).str[1]

In [137]:
restaurantes["Almoco"] = restaurantes["Almoco"].fillna('').str.split(' - Jantar:',n=1).str[0]

In [138]:
restaurantes["Jantar"] = restaurantes["Valor"].fillna('').str.split(' - Jantar: ',n=1).str[1]

In [139]:
restaurantes["Menu"] = restaurantes["Valor"].fillna('').str.split('Menu ',n=1).str[1]

In [140]:
restaurantes["Menu"] = restaurantes["Menu"].fillna('').str.split(' -',n=1).str[0]

In [141]:
restaurantes = restaurantes[['Nome', 'Cozinha', 'Bairro', 'Almoco', 'Jantar', 'Menu']]

In [142]:
##filtro = restaurantes['Menu'] == 'RW'
##restaurantesF = restaurantes[filtro]

In [143]:
restaurantes

,Nome,Cozinha,Bairro,Almoco,Jantar,Menu
0,El Carreiro,"Argentina, Parrilla",Stella Maris,"R$ 73,90","R$ 94,90",+Plus
1,La Lupa,"Italiana, Mediterrânea",Santo Antônio Além do Carmo,"R$ 73,90","R$ 94,90",+Plus
2,Cedro - Quality Hotel e Suítes São Salvador,Contemporânea,Stiep,"R$ 73,90","R$ 94,90",+Plus
3,Canto Restaurante,Internacional,Rio Vermelho,"R$ 95,00","R$ 115,00",Premium
4,Joca Mesa Bar,"Contemporânea, Regional",Santo Antônio,"R$ 73,90","R$ 94,90",+Plus
...,...,...,...,...,...,...
458,Pereira Restaurante - Porto da Barra,Contemporânea,Barra,"R$ 95,00","R$ 115,00",Premium
459,Pasta em Casa - Alphaville,Italiana,Alphaville I,"R$ 95,00","R$ 115,00",Premium
460,Cantina Volpi - Paralela,Italiana,Alphaville I,"R$ 73,90",NaN,+Plus
461,Seven Wonders Café - Spot Barra,,Barra,"R$ 59,90",NaN,RW


In [144]:
temp1 = p.DataFrame(restaurantes['Cozinha1'].drop_duplicates())
temp1['Cozinha'] = temp1['Cozinha1']
temp1 = temp1.drop('Cozinha1', axis=1)
temp2 = p.DataFrame(restaurantes['Cozinha2'].drop_duplicates())
temp2['Cozinha'] = temp2['Cozinha2']
temp2 = temp2.drop('Cozinha2', axis=1)
temp = p.concat([temp1, temp2], ignore_index=True)
temp = p.DataFrame(temp['Cozinha'].drop_duplicates())
temp

# Suponha que você tenha duas Series: serie1 e serie2
    #  df_temp = pd.DataFrame({'coluna1': serie1, 'coluna2': serie2})
    #  df_concatenado = pd.concat([df_original, df_temp], axis=1)


KeyError: 'Cozinha1'

In [145]:
temp = p.DataFrame(restaurantes['Bairro'].drop_duplicates())
temp = temp.reset_index()
temp

,index,Bairro
0,0,Stella Maris
1,1,Santo Antônio Além do Carmo
2,2,Stiep
3,3,Rio Vermelho
4,4,Santo Antônio
5,7,Vilas do Atlântico
6,8,São Cristóvão
7,11,A Rio Vermelho
8,27,Pituba
9,29,NaN


In [ ]:
# Dados iniciais
data = {'coluna1': [1, 2, 3], 'coluna2': ['a', 'b', 'c']}
df = p.DataFrame(data)

# Novos dados
novos_dados = {'coluna1': [4, 5], 'coluna2': ['d', 'e']}
df_novos = p.DataFrame(novos_dados)

# Adiciona os novos dados ao final do DataFrame original
df = p.concat([df, df_novos], ignore_index=True)

df

,coluna1,coluna2
0,1,a
1,2,b
2,3,c
3,4,d
4,5,e
